<a href="https://colab.research.google.com/github/tomonari-masada/course2022-nlp/blob/main/08_document_classification_with_RNN_(in_class).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNNを使った文書分類
* RNNの出力を文書の潜在表現として利用し、文書分類を行う。
* 単語の埋め込みも含めて学習する。

## 準備
* ランタイムのタイプをGPUにしておこう。


* 再現性の確保については下記を参照。
 * https://pytorch.org/docs/stable/notes/randomness.html

In [1]:
import time
from collections import OrderedDict
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
#from torchtext.vocab import vocab # ここに書いておくとエラーが出る？？？

np.random.seed(123)
torch.manual_seed(123)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
device

device(type='cuda')

## IMDbデータの準備
* IMDbデータセットは`torchtext.datasets`から使うこともできる。
 * https://pytorch.org/text/stable/datasets.html
 * https://torchtext.readthedocs.io/en/latest/datasets.html
* だが、語彙集合を作成するために使う`torchtext.vocab.build_vocab_from_iterator`という関数がとても遅い・・・。
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
* なので、ここではCountVectorizerで語彙集合を作ることにした。

In [3]:
!pip install ml_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from ml_datasets import imdb

train_data, test_data = imdb()
train_texts, train_labels = zip(*train_data)
test_texts, test_labels = zip(*test_data)

# ラベルは整数の1と0に変換しておく
label_id = { "pos":1, "neg":0 }

train_labels = [label_id[label] for label in train_labels]
test_labels = [label_id[label] for label in test_labels]

84131840it [00:08, 9410037.95it/s]                               


Untaring file...


In [5]:
train_texts[0]

"Tight script, good direction, excellent performances, strong cast, effective use of locations....\n\n\n\nPaul McGann gives a detailed, subtle performance as the man in the centre of a new murder investigation who may just have committed a similar murder previously.\n\n\n\nThere is an interesting moral & emotional journey happening with his character (Ben Turner) and it intersects with the journey undertaken by Amanda Burton. Inevitably they cross over... Who has done what?\n\n\n\nThe examination of WHY, both in the past and in the present, rather than WHO might have yielded a more interesting, Dostoyevskian story, but hey, who's complaining?\n\n\n\n"

In [6]:
train_labels[0]

1

## sklearnのCountVectorizerによるトークン化
* 一般に、語彙集合を確定させるときは、訓練データだけを使う。

### CountVectorizerで語彙集合を作成
* ここで作った語彙集合のサイズを、あとで絞る。
* `CountVectorizer`の`token_pattern`を指定し、1文字の単語が消えないようにする。
 * ここでは、テキストを、bag-of-wordsとしてではなく、単語の列としてモデル化する。
 * その場合、例えば冠詞"a"が消えてしまうのは避けたい。
* ただし、下の正規表現だと、punctuationは消えてしまう。

In [7]:
# デフォルトの正規表現とは異なる正規表現を使う
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")

# 訓練データのテキストをトークン化する
X = vectorizer.fit_transform(train_texts)

# 語彙集合を取得する
vocabulary = vectorizer.get_feature_names_out()
print(len(vocabulary))

74891


### 単語のdocument frequencyを計算
* このままだと語彙サイズが大きすぎる。
* 後で、語彙を絞り込むために、document frequencyを使うことにする。
 * document frequencyが小さい単語は、未知語として扱うことにする。

In [8]:
doc_freq = np.array((X > 0).sum(0)).squeeze()

* document frequencyでトップ10の単語を見てみる。

In [9]:
print(vocabulary[np.argsort(- doc_freq)][:10])

['the' 'a' 'and' 'of' 'to' 'this' 'is' 'it' 'in' 'that']


## `torchtext`の語彙集合の作成
* 作り方は下記のリンク先を参照。
 * https://pytorch.org/text/stable/vocab.html#id1

### OrderedDictを作成
 * torchtextで語彙集合を作成するとき、OrderedDictを渡すことになっているため。

In [10]:
vocab_ordered_dict = OrderedDict(zip(vocabulary, doc_freq))
len(vocab_ordered_dict)

74891

* 適当な単語のdocument frequencyを見てみる。

In [11]:
vocab_ordered_dict["apple"]

48

### `torchtext`の語彙集合を作成
* ここでは、document frequencyが10未満の単語を未知語とすることで、語彙サイズを抑えている。
 * ここはチューニングする余地がある。

In [14]:
from torchtext.vocab import vocab

# 未知語は全て"<unk>"という特殊なトークンへ置き換えることにする
unknown_token = "<unk>"

# padding用のトークンを作っておく
padding_token = "<pad>"

# OrderedDictをもとにtorchtextでの語彙集合を作成
#   min_freqを指定すると、低頻度語は全て未知語として扱われる。
#   ここで、OrderedDictの各keyに対応するvalueが用いられる。
#   （つまり、document frequency以外の値で未知語を決めても構わない。）
vocab = vocab(
    vocab_ordered_dict, min_freq=10,
    specials=[unknown_token, padding_token],
    )

# 語彙にない単語のインデックスは全て"<unk>"と同じインデックスになるよう、設定する
vocab.set_default_index(vocab[unknown_token])

print(len(vocab))

18563


## テキストを単語idの列へ変換する関数

In [15]:
# fit済みのCountVectorizerから、前処理とトークナイザを持ってくる
preprocessor = vectorizer.build_preprocessor()
tokenizer = vectorizer.build_tokenizer()

# 前処理、トークナイザ、インデックス列への変換を、一つの処理としてまとめる
text_pipeline = lambda x: vocab(tokenizer(preprocessor(x)))

* このトークナイザでトークン化すると、punctuationは消えることに注意。

In [16]:
text = "This is a fnekjfjd."
print(preprocessor(text))
print(tokenizer(preprocessor(text)))
print(text_pipeline(text))

this is a fnekjfjd.
['this', 'is', 'a', 'fnekjfjd']
[16665, 8906, 272, 0]


* 特殊なトークンには、以下のインデックスが割り振られている。

In [17]:
vocab.get_stoi()['<pad>']

1

In [18]:
vocab.get_stoi()['<unk>']

0

* インデックスから単語への変換は、以下のように行うことができる。

In [19]:
vocab.get_itos()[1001]

'apple'

## Dataset

### 全テキストのトークン化
* 単語インデックスの列への変換から、テンソルへの変換まで、おこなっている。

In [20]:
train_tokens = [torch.tensor(text_pipeline(text), dtype=torch.int64) for text in train_texts]
test_tokens = [torch.tensor(text_pipeline(text), dtype=torch.int64) for text in test_texts]

In [21]:
print(train_tokens[0])

tensor([16755, 14467,  7232,  4810,  5922, 12107, 15926,  2682,  5434, 17587,
        11473,  9825, 12012,     0,  7139,   272,  4651, 16030, 12106,  1149,
        16598, 10126,  8430, 16598,  2785, 11473,   272, 11190, 10952,  8851,
        18178, 10332,  9166,  7711,  3403,   272, 14973, 10952, 12736, 16629,
         8906,   819,  8755, 10816,  5574,  9111,  7623, 18292,  7959,  2852,
         1724, 17184,   843,  8930,     0, 18292, 16598,  9111,     0,  2446,
          760,  2407,  8547, 16643,  4053, 11720, 18178,  7681,  5062, 18130,
        16598,  5909, 11473, 18196,  2102,  8430, 16598, 11968,   843,  8430,
        16598, 12684, 13259, 16587, 18178, 10558,  7711,     0,   272, 10826,
         8755,     0, 15845,  2429,  7893, 18178, 14194,  3458])


### 自前のデータセットの定義

In [22]:
class MyTextDataset(Dataset):
  def __init__(self, labels, tokens):
    self.labels = labels
    self.tokens = tokens

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, index):
    return self.labels[index], self.tokens[index]

train_dataset = MyTextDataset(train_labels, train_tokens)
test_dataset = MyTextDataset(test_labels, test_tokens)

### validation setの切り分け

In [23]:
valid_size = len(train_dataset) // 5
train_size = len(train_dataset) - valid_size
test_size = len(test_dataset)

split_train_, split_valid_ = random_split(train_dataset, [train_size, valid_size])

## DataLoader

### RNNの入力として使えるミニバッチを作る関数を定義
* paddingして、同じミニバッチに含まれる単語id列の長さを揃える関数。
* この関数は、DataLoaderクラスのインスタンスを作るときに、`collate_fn`の値として指定する。

In [24]:
# paddingに使うトークンのインデックスを取得
PAD_IDX = vocab.get_stoi()[padding_token]

def collate_batch(batch):
  labels, tokens = zip(*batch)
  labels = torch.tensor(labels, dtype=torch.int64)
  tokens = pad_sequence(tokens, padding_value=PAD_IDX)
  return labels, tokens

* 試しにバッチサイズ4でDataLoaderを作って、ミニバッチの中身を見てみる。

In [25]:
BATCH_SIZE = 4
train_loader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

In [26]:
labels, tokens = next(iter(train_loader))
print(labels)
print(tokens.shape)
print(tokens)

tensor([0, 0, 1, 1])
torch.Size([729, 4])
tensor([[16598, 16665,  2685, 16665],
        [    0,  8554, 11473,  8906],
        [ 8906,  5630,  1957, 11543],
        ...,
        [    1, 16806,     1,     1],
        [    1,  7959,     1,     1],
        [    1, 14600,     1,     1]])


* ミニバッチのサイズは64にしておく。
 * ここはチューニングできる。

In [27]:
BATCH_SIZE = 64
train_loader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
valid_loader = DataLoader(split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

## モデル
* LSTMを使う（GRUに変えても良い）
 * http://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [28]:
VOCAB_SIZE = len(vocab)
NUM_CLASS = 2
EMSIZE = 64
HID_DIM = 64

In [29]:
class RNNTextSentiment(nn.Module):
  def __init__(self, emb_dim, hid_dim,
               num_class, vocab_size, padding_idx, p=0.0):
    super().__init__()

    self.input_dim = vocab_size
    self.emb_dim = emb_dim
    self.hid_dim = hid_dim
    self.dropout = p

    self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=padding_idx)
    self.rnn = nn.LSTM(emb_dim, hid_dim)
    self.fc = nn.Linear(hid_dim * 2, num_class)
    self.dropout = nn.Dropout(p=p)

  def forward(self, src):
    # embeddedの形は(トークン列長, バッチサイズ, 埋め込み次元数)
    embedded = self.dropout(self.embedding(src))

    # outputsの形は(トークン列長, バッチサイズ, 隠れ状態の次元数)
    # hiddenの形は(1, バッチサイズ, 隠れ状態の次元数)
    outputs, (hidden, _) = self.rnn(embedded)

    # mean_outputsの形は(バッチサイズ, 隠れ状態の次元数)
    # hiddenの形は(バッチサイズ, 隠れ状態の次元数)
    mean_outputs = outputs.mean(0)
    hidden = hidden.squeeze()

    return self.fc(torch.cat((mean_outputs, hidden), dim=1))

In [30]:
model = RNNTextSentiment(
    EMSIZE, HID_DIM, NUM_CLASS, VOCAB_SIZE,
    padding_idx=PAD_IDX,
    p=0.5,
    ).to(device)

## 最適化アルゴリズム

In [31]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

* パラメータの数を数えてみる。

In [32]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters.')

The model has 1,221,570 trainable parameters.


* token embeddingに使うパラメータの個数だけを数えてみる。

In [35]:
def count_embedding_parameters(model):
  return sum(p.numel() for p in model.embedding.parameters() if p.requires_grad)

print(f'The model has {count_embedding_parameters(model):,} trainable embedding parameters.')

The model has 1,188,032 trainable embedding parameters.


## 損失関数

In [33]:
criterion = nn.CrossEntropyLoss()

## 訓練を行なう関数

In [34]:
def train(dataloader, clip=1.):
  model.train()
  total_loss, total_acc, total_count = 0, 0, 0
  for label, text in dataloader:
    label, text = label.to(device), text.to(device)
    output = model(text)
    loss = criterion(output, label)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    optimizer.zero_grad()
    n_instances = label.size(0)
    total_loss += loss.item() * n_instances
    total_acc += (output.argmax(1) == label).sum().item()
    total_count += n_instances
  return total_loss / total_count, total_acc / total_count

## 評価を行なう関数

In [36]:
def evaluate(dataloader):
  model.eval()
  total_loss, total_acc, total_count = 0, 0, 0
  for label, text in dataloader:
    label, text = label.to(device), text.to(device)
    with torch.no_grad():
      output = model(text)
      loss = criterion(output, label)
      n_instances = label.size(0)
      total_loss += loss.item() * n_instances
      total_acc += (output.argmax(1) == label).sum().item()
      total_count += n_instances
  return total_loss / total_count, total_acc / total_count

## 時間表示用の関数

In [37]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time // 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

## 学習の実行

In [38]:
N_EPOCHS = 10

for epoch in range(1, N_EPOCHS + 1):

  start_time = time.time()
  train_loss, train_acc = train(train_loader)
  valid_loss, valid_acc = evaluate(valid_loader)
  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch {epoch} | time in {epoch_mins} minutes, {epoch_secs} seconds')
  print(f'\tLoss {train_loss:.4f} (train)\t|\tAcc {train_acc*100:.1f}% (train)')
  print(f'\tLoss {valid_loss:.4f} (valid)\t|\tAcc {valid_acc*100:.1f}% (valid)')

Epoch 1 | time in 0 minutes, 4 seconds
	Loss 0.6749 (train)	|	Acc 57.1% (train)
	Loss 0.6183 (valid)	|	Acc 68.1% (valid)
Epoch 2 | time in 0 minutes, 3 seconds
	Loss 0.5938 (train)	|	Acc 69.4% (train)
	Loss 0.5248 (valid)	|	Acc 76.1% (valid)
Epoch 3 | time in 0 minutes, 2 seconds
	Loss 0.5281 (train)	|	Acc 75.5% (train)
	Loss 0.6024 (valid)	|	Acc 71.0% (valid)
Epoch 4 | time in 0 minutes, 2 seconds
	Loss 0.4799 (train)	|	Acc 79.4% (train)
	Loss 0.5898 (valid)	|	Acc 76.9% (valid)
Epoch 5 | time in 0 minutes, 2 seconds
	Loss 0.4330 (train)	|	Acc 81.2% (train)
	Loss 0.4174 (valid)	|	Acc 83.2% (valid)
Epoch 6 | time in 0 minutes, 2 seconds
	Loss 0.4063 (train)	|	Acc 82.7% (train)
	Loss 0.4175 (valid)	|	Acc 83.4% (valid)
Epoch 7 | time in 0 minutes, 3 seconds
	Loss 0.3643 (train)	|	Acc 84.7% (train)
	Loss 0.4024 (valid)	|	Acc 84.6% (valid)
Epoch 8 | time in 0 minutes, 3 seconds
	Loss 0.3416 (train)	|	Acc 85.8% (train)
	Loss 0.4610 (valid)	|	Acc 84.2% (valid)
Epoch 9 | time in 0 minutes, 3 s

# 課題6
* 上のコードを動かして、感情分析を実践してみよう。
* 余裕があれば、ハイパーパラメータをチューニングして、分類性能を上げてみよう。
 * 例えば、LSTMのレイヤ数を2以上にすると、性能は上がるだろうか？

## テストデータで評価

In [ ]:
loss, acc = evaluate(test_loader)
print(f'\tLoss {loss:.4f} (test)\t|\tAcc {acc*100:.1f}% (test)')